In [31]:
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df_credit = pd.read_csv('../data/credit_card.csv')
df_credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df_credit['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [8]:
#separeting the fraud and non fraud data
legit = df_credit[df_credit.Class == 0]
fraud = df_credit[df_credit.Class == 1]


In [9]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [10]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [11]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [12]:
##compare the values for both transactions
df_credit.groupby('Class').mean()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [13]:
legit_sample = legit.sample(n=492)

In [14]:
new_dataset = pd.concat([legit_sample,fraud], axis=0)

In [15]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
143641,85507.0,-0.395559,1.038879,1.203863,-0.287007,0.598547,0.072363,0.667104,0.077780,-0.469565,...,-0.245927,-0.565452,-0.106731,-0.762627,-0.126516,0.150100,0.286289,0.110866,2.69,0
237943,149461.0,-1.096401,0.632418,1.670101,-0.025553,-0.171099,0.185873,-0.303729,0.646746,-0.265961,...,0.306181,0.823680,-0.495328,-0.277173,0.603836,0.089087,-0.019539,-0.046429,9.99,0
46095,42638.0,-1.160903,1.151422,-0.918765,0.872705,-0.085144,-0.277674,0.202334,0.899959,-0.761588,...,0.221992,0.617456,0.039441,-0.309553,-0.723957,-0.370495,0.412846,0.013050,89.99,0
223638,143464.0,-1.245689,1.093125,0.760991,-2.539969,0.425615,-0.618822,0.823957,0.336239,-0.166650,...,-0.036290,-0.115706,-0.363072,0.688513,0.868905,0.738597,0.024925,0.031489,19.20,0
252436,155806.0,-0.069901,0.668813,0.332842,-0.480379,0.817013,-0.585782,0.878563,-0.112890,-0.579305,...,-0.102229,-0.255900,0.074531,-0.319131,-1.004568,0.136806,0.157141,0.205588,7.27,0


In [16]:
new_dataset['Class'].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [17]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,96484.420732,0.035440,-0.022304,-0.017874,0.041662,0.056582,0.015380,-0.007094,0.010397,0.035981,...,0.013445,0.049630,-0.030932,-0.002975,0.001004,-0.001470,0.010290,0.033042,-0.014181,86.110020
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [18]:
##Splitting the data into Features and Targets
X = new_dataset.drop(columns='Class',  axis=1)
Y = new_dataset['Class']


In [19]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
143641   85507.0 -0.395559  1.038879  1.203863 -0.287007  0.598547  0.072363   
237943  149461.0 -1.096401  0.632418  1.670101 -0.025553 -0.171099  0.185873   
46095    42638.0 -1.160903  1.151422 -0.918765  0.872705 -0.085144 -0.277674   
223638  143464.0 -1.245689  1.093125  0.760991 -2.539969  0.425615 -0.618822   
252436  155806.0 -0.069901  0.668813  0.332842 -0.480379  0.817013 -0.585782   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [20]:
print(Y)

143641    0
237943    0
46095     0
223638    0
252436    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [21]:
##Split the data into Traning data and Testing Data

X_train, X_test, Y_train,Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [22]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


In [23]:
model = LogisticRegression()

In [24]:
# training the Logistic Regression Model with Training Data 
model.fit(X_train, Y_train)

c:\Users\hp\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
##Model Evaluation 
#Accuracy Score
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [28]:
print('Accuracy on Training data :', training_data_accuracy)

Accuracy on Training data : 0.9466327827191868


In [29]:
# accuracy on test data 
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test) 

In [30]:
print('Accuracy score on Test Data :', test_data_accuracy)

Accuracy score on Test Data : 0.949238578680203


In [39]:
mlflow.sklearn.autolog()

In [40]:
with mlflow.start_run(run_name='credit fraud'):

    model = LogisticRegression(max_iter=100)
    model.fit(X_train, Y_train)

    predictions = model.predict(X_test)

    accuracy = accuracy_score(Y_test, predictions)
    print(f'Accuracy: {accuracy}')
    
    mlflow.log_metric('accuracy', accuracy)

    mlflow.sklearn.log_model(model, 'logistic_regression_model')

    mlflow.log_param('max_iter', 100)
    mlflow.log_param('solver', 'lbfgs')

    #run_id = mlflow.active_run().info.run_id
    #print(f'Run ID: {run_id}')

c:\Users\hp\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.949238578680203


2025/02/13 16:27:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
mlflow.set_experiment('credit fraud')

<Experiment: artifact_location='file:///c:/Users/hp/kifiya_acadamy_week8%269/kifiya_week8-9_assignment/notebooks/mlruns/978457570631271889', creation_time=1739449887734, experiment_id='978457570631271889', last_update_time=1739449887734, lifecycle_stage='active', name='credit fraud', tags={}>